avaliando com o método da matriz de confusão que foi ensinado em aula (Métrica principal)

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

# ----------------- Config -----------------
PATH_USUARIOS = r"D:/Dev/projetos vscode/SuggestAI/data/usuarios_filmes.csv"
PATH_FILMES = r"D:/Dev/projetos vscode/SuggestAI/data/filmes.csv"

# ----------------- Leitura dos CSVs -----------------
df_usuarios = pd.read_csv(PATH_USUARIOS)
df_filmes = pd.read_csv(PATH_FILMES)

df_usuarios.columns = [c.strip() for c in df_usuarios.columns]
df_filmes.columns = [c.strip() for c in df_filmes.columns]

# ----------------- Colunas de gêneros -----------------
generos_cols = ['acao','comedia','drama','ficcao cientifica','romance','terror','suspense']

# ----------------- Criar estrutura de usuários -----------------
usuarios = {}
for _, row in df_usuarios.iterrows():
    filmes = [f for f in row[['filme1','filme2','filme3','filme4','filme5','filme6','filme7']] if pd.notna(f) and f.strip() != '']
    genero_scores = row[generos_cols].fillna(0).astype(float).to_dict()
    usuarios[row['user_id']] = {"movies": filmes, **genero_scores}

# ----------------- Divisão treino/teste -----------------
for uid, info in usuarios.items():
    filmes = info["movies"]
    if len(filmes) <= 1:
        usuarios[uid]["train"] = filmes
        usuarios[uid]["teste"] = []
    else:
        train, teste = train_test_split(filmes, test_size=0.4, random_state=42)
        usuarios[uid]["train"] = train
        usuarios[uid]["teste"] = teste

# ----------------- Classe Recomendador -----------------
class InteractiveRecommender:
    def __init__(self, usuarios, filmes_df):
        self.users = usuarios
        self.filmes_df = filmes_df

    def get_recommendations_inteligente(self, user_id, top_n=3):
        filmes_vistos = set(self.users[user_id]["train"])
        perfil_genero = np.array([self.users[user_id].get(g,0) for g in generos_cols], dtype=float)
        all_films = list(self.filmes_df['filme'].values)

        scores = {}
        for filme in all_films:
            if filme in filmes_vistos:
                continue
            genero = self.filmes_df.loc[self.filmes_df['filme']==filme,'genero'].values[0]
            vec_genero = np.array([1 if g==genero else 0 for g in generos_cols], dtype=float)
            score = np.dot(perfil_genero, vec_genero)
            scores[filme] = score

        recs = sorted(scores, key=lambda x: scores[x], reverse=True)[:top_n]
        return recs

    # Placeholders
    def recommend_by_knn(self, user_id, top_n=3):
        return self.get_recommendations_inteligente(user_id, top_n)

    def recommend_by_weights(self, user_id, top_n=3):
        return self.get_recommendations_inteligente(user_id, top_n)

    def recommend_by_matrix_factorization(self, user_id, top_n=3):
        return self.get_recommendations_inteligente(user_id, top_n)

# ----------------- Instancia o recomendador -----------------
recommender = InteractiveRecommender(usuarios, df_filmes)

# ----------------- Função de avaliação combinando todos os métodos -----------------
def avaliar_recomendador_combinado(recommender, top_n=3):
    y_true, y_pred = [], []

    for uid, dados in usuarios.items():
        if not dados["teste"]:
            continue

        # Combinar recomendações de todos os métodos
        recs_combined = set()
        recs_combined.update(recommender.get_recommendations_inteligente(uid, top_n=top_n))
        recs_combined.update(recommender.recommend_by_knn(uid, top_n=top_n))
        recs_combined.update(recommender.recommend_by_weights(uid, top_n=top_n))
        recs_combined.update(recommender.recommend_by_matrix_factorization(uid, top_n=top_n))

        # True Positives e False Negatives
        for filme in dados["teste"]:
            y_true.append(1)
            y_pred.append(1 if filme in recs_combined else 0)

        # False Positives
        for filme in recs_combined:
            if filme not in dados["teste"]:
                y_true.append(0)
                y_pred.append(1)

    cm = confusion_matrix(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)

    return cm, prec, rec, f1

# ----------------- Avaliação -----------------
cm, prec, rec, f1 = avaliar_recomendador_combinado(recommender, top_n=3)
print("\n=== Avaliação Combinando Todos os Métodos ===")
print("Matriz de Confusão:\n", cm)
print(f"Precisão: {prec:.2f}")
print(f"Recall: {rec:.2f}")
print(f"F1-Score: {f1:.2f}")


C:\Users\USER\AppData\Local\Temp\ipykernel_1088\3928851143.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  genero_scores = row[generos_cols].fillna(0).astype(float).to_dict()
C:\Users\USER\AppData\Local\Temp\ipykernel_1088\3928851143.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  genero_scores = row[generos_cols].fillna(0).astype(float).to_dict()
C:\Users\USER\AppData\Local\Temp\ipykernel_1088\3928851143.py:24: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call resu


=== Avaliação Combinando Todos os Métodos ===
Matriz de Confusão:
 [[  0 137]
 [ 82  79]]
Precisão: 0.37
Recall: 0.49
F1-Score: 0.42


Gráfico de cobertura de filmes (Gráfico de dados como recomendado)